# Day 16

A graph traversal problem with a twist.
Goal here is not minimisation, but maximisation.

Initial thoughts:
1) Look at the input file this time and get a sense of how big the problem is.... it's get 57 valves. So not huge.
2) Might be able to build up a tree of all the options... I'm not sure a classic Dijkstra's algo will work here, as the "value" of visiting a node changes with time.
3) But still wouldn't want to go back to a node after visiting it




In [ ]:
testData = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
"""

import re
import pprint
import copy

import multiprocess as mp

# regex primer:
# pattern = '.*x=(-*[0-9]*).*y=(-*[0-9]*).*closest beacon is at x=(-*[0-9]*).*y=(-*[0-9]*)'
# match = re.search(pattern, l)
# sensorPos = (int(match.group(1)),int(match.group(2)))

class Valve:
    def __init__(self, valve, flow, paths):
        self.valve = valve
        self.flow = flow
        self.paths = paths

class Scenario:
    #contains the result and logic for a scenario
    def __init__(self, network):
        self.network = network
        #for a fresh scenario (note: will end up with a lot of scenarios being a deep copy, so they won't reinitalise)
        self.time = 0
        self.currentValve:Valve = self.network.valves['AA']
        self.openValves:[Valve] = [] 
        self.openValveIdentifiers = []
        self.cummulativeFlow = 0

    def key(self)->str:
        #create a composite key of all the important bit of the scenario, so we can de-dupe
        self.openValves.sort(key=lambda x: x.valve)
        output = str(self.cummulativeFlow) + '-' + str(self.currentValve.valve) + '-'
        for valve in self.openValves:
            output = output + str(valve.valve) + ':'
        return output


    def clockTick(self)->[]:
        #the scenario tick on a click. Returns multiple realities of scenario. Note: type missing in return because python 3 type system is... weird.
        self.time += 1

        #update the cummulative flow with all open valves
        for valve in self.openValves:
            self.cummulativeFlow += valve.flow

        output:[] = []
        #open valve or move?
        if not (self.currentValve.flow == 0 or self.currentValve.valve in self.openValveIdentifiers):
            #opening valve is a valid move
            newScenario = copy.deepcopy(self)
            newScenario.openValves.append(self.currentValve)
            newScenario.openValveIdentifiers.append(self.currentValve.valve)
            output.append(newScenario)
        
        #is is possible that it's better to skip opening a valve an move on? Don't know... so let's include it in an option
        for path in self.currentValve.paths:
            newScenario = copy.deepcopy(self)
            newScenario.currentValve = self.network.valves[path]
            output.append(newScenario)

        return output



class Network:

    def __init__(self,input:str):
        pattern = 'Valve ([A-Z][A-Z]) has flow rate=([0-9]*); tunnels? leads? to valves? (.*)$'
        self.valves: dict[str,tuple[int,list[str]]] = {} #dictionary of valve -> (flow, [paths])
        for l in input.splitlines():
            match = re.search(pattern,l)
            if match:
                valveName = match.group(1)
                flow = int(match.group(2))
                paths = match.group(3).split(', ')
                valve = Valve(valveName,flow,paths)
                self.valves[valveName] = valve
        #pprint.pprint(self.valves)
    
    def runScenarios(self):
        #simple brute force strategy... whenever there is an option, we fork time.
        forks = {} #this is where we hold all our version of state
        #we always start at minute 1 and run to minute 30
        time = 0
        rootScenario = Scenario(self)
        forks[rootScenario.key()] = rootScenario
        bestScenarioFlow = 0

        while time < 30:
            time += 1
            print('Time t='+str(time)+' Processing forks:'+str(len(forks.keys())))
            #This approach is leading to too many scenarios... we need to cull some as we go.
            #Would the winning scenario always be ahead of the others? Maybe not, but it won't be far behind... but how much.... hmmm... how about 50... try a configurable threshold?
            for scenario in forks.values():
                bestScenarioFlow = max(bestScenarioFlow,scenario.cummulativeFlow)
            threshold = bestScenarioFlow * (7/10) - 20
            print('Best scenario flow='+str(bestScenarioFlow) + '  Setting threshold at:'+str(threshold))
            
            newForks = {}

            
            for scenario in forks.values():
                if scenario.cummulativeFlow >= threshold:
                    for sc in scenario.clockTick():
                        key = sc.key()
                        print(key)
                        newForks[key] = sc
            
            # print('Let\'s hack time... using ' + str(mp.cpu_count())+ ' CPUs')
            # with mp.Pool(mp.cpu_count()) as pool:
            #     for scenario in forks:
            #         if scenario.cummulativeFlow >= threshold:
            #             newForks.extend(pool.apply_async(scenario.clockTick())) #TODO: FIX THIS... it doens't seem to work because apply_asyn is horrible.
            #     pool.join()
            

            forks = newForks

        #we should now have all the scenarios possible in our forks
        print('---')
        print('Total forks: ' + str(len(forks)))
        finalScenarios = list(forks.values())
        finalScenarios.sort(key=lambda x: x.cummulativeFlow, reverse=True)
        winningScenario = finalScenarios[0]
        print('Max Flow: ' + str(winningScenario.cummulativeFlow))
        print(winningScenario.key())





#test data
testnet = Network(testData)
testnet.runScenarios()


We're still ending up with a lot of forks, even when we're setting the threshold of "currently the best"... why?

How many of our forked scenarios are effectively the same? Quick approch to check: stick them in a dictionary rather than array, with a suitable composite key. Yeah, that's made it much better!

New problem: we're geting an answer of 2222. Test data is giving answeer of 1651.
Soln: deep copy, and then fix checking for membership - as no longer same objects.




In [ ]:
#run with real input
data = open('day16input.txt').read()
net = Network(data)
net.runScenarios()
